**Import libraries**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from random import randint

import eli5
import eli5.sklearn.explain_prediction as expred
import eli5.sklearn.explain_weights as exweight
import eli5.xgboost as xgb
import eli5.lightgbm as lgbm
import eli5.catboost as catb
from pdpbox import pdp, get_dataset, info_plots
from yellowbrick.regressor import ResidualsPlot
from yellowbrick.regressor import PredictionError
from yellowbrick.regressor import AlphaSelection
from yellowbrick.regressor import CooksDistance
#from yellowbrick.classifier import *
import shap

from pycaret.regression import *
from pycaret.classification import *
import explainer as ex_intr

D:\Users\ajoshi\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
D:\Users\ajoshi\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
%matplotlib inline

In [19]:

class Interpreter:
    def __init__(self, package):

        # data unloading
        self.model = package[0]
        self.X_train = package[1]
        self.X_test = package[2]
        self.y_train = package[3]
        self.y_test = package[4]
        self.base_features = package[5]
        self.model_type = str(str(type(self.model)).split('.')[-1].split("'")[0])
        print(self.model_type)
        self.regressors = ['LinearRegression', 'Ridge', 'RandomForestRegressor', 'DecisionTreeRegressor', 'XGBRegressor',
                           'LGBMRegressor', 'CatBoostRegressor']
        self.classifiers = ['LogisticRegression', 'LogisticRegressionCV', 'RidgeClassifier','RandomForestClassifier', 
                            'DecisionTreeClassifier', 'QuadraticDiscriminantAnalysis']

    def interprete_model(self, interpretation_tool, **kwargs):
        
        if(self.model_type in self.regressors):
            print("model == Random Forest")
            plot_type = kwargs.get('plotname', 'residuals')
        elif(self.model_type in self.classifiers):
            plot_type = kwargs.get('plotname', 'auc')

        print('Model name: ' + str(self.model_type))
        # for linear regressors
        if (self.model_type in self.regressors) and interpretation_tool == 'eli5':  # for eli5
            print('\n\nPlotting ELI5 interpretations...\n\n')
            if (self.model_type == 'LinearRegression') or (self.model_type == 'Ridge'):

                ex_intr.Explainer('EPLR')
                display(
                    expred.explain_prediction_linear_regressor(self.model, doc=self.X_train.values[randint(0, 100)]))

                ex_intr.Explainer('EPLRW')
                display(exweight.explain_linear_regressor_weights(self.model))

                # explain permutation importance
                perm = eli5.sklearn.PermutationImportance(self.model, random_state=1).fit(self.X_train, self.y_train)
                ex_intr.Explainer('ESW')
                display(eli5.show_weights(perm, feature_names=self.X_train.columns.tolist()))

            elif self.model_type == 'RandomForestRegressor':
                # explain tree regressor feature importance
                
                # show predictions
                display(eli5.show_prediction(self.model, self.X_test.iloc[10], show_feature_values=True))

                ex_intr.Explainer('EPTR')
                display(expred.explain_prediction_tree_regressor(self.model, doc=self.X_train.values[randint(0, 100)],
                                                                 feature_names=self.X_train.columns.tolist()))

                # explain feature importance without permutation
                ex_intr.Explainer('EFRF')
                display(exweight.explain_rf_feature_importance(self.model, feature_names=self.X_train.columns.tolist()))

                # explain with permutation, the feature importance
                perm = eli5.sklearn.PermutationImportance(self.model, random_state=1).fit(self.X_train, self.y_train)
                ex_intr.Explainer('ESW')
                display(eli5.show_weights(perm, feature_names=self.X_train.columns.tolist()))

            elif self.model_type == 'XGBRegressor':
                print('Hello')
                display(xgb.explain_weights_xgboost(self.model))

            elif self.model_type == 'LGBMRegressor':
                print('Hello')
                display(lgbm.explain_weights_lightgbm(self.model))

            elif self.model_type == 'CatBoostRegressor':
                display(catb.explain_weights_catboost(self.model))


        elif (self.model_type in self.classifiers) and interpretation_tool == 'eli5':
            if self.model_type == 'LogisticRegression' or self.model_type == 'QuadraticDiscriminantAnalysis':
                display(
                    expred.explain_prediction_linear_classifier(self.model, doc=self.X_train.values[randint(0, 100)]))
                display(exweight.explain_linear_classifier_weights(self.model))
                # explain permutation importance
                perm = eli5.sklearn.PermutationImportance(self.model, random_state=1).fit(self.X_train, self.y_train)
                display(eli5.show_weights(perm, feature_names=self.X_train.columns.tolist()))

            elif self.model_type == 'DecisionTreeClassifier':
                display(expred.explain_prediction_tree_classifier(self.model, doc=self.X_train.values[randint(0, 100)]))
                display(exweight.explain_decision_tree(self.model))
                # explain permutation importance
                perm = eli5.sklearn.PermutationImportance(self.model, random_state=1).fit(self.X_train, self.y_train)
                display(eli5.show_weights(perm, feature_names=self.X_train.columns.tolist()))

            elif self.model_type == 'RandomForestClassifier':
                display(expred.explain_prediction_tree_classifier(self.model, doc=self.X_train.values[randint(0, 100)]))
                display(exweight.explain_rf_feature_importance(self.model))
                display(eli5.show_weights(self.model))
                # explain permutation importance
                perm = eli5.sklearn.PermutationImportance(self.model, random_state=1).fit(self.X_train, self.y_train)
                display(eli5.show_weights(perm, feature_names=self.X_train.columns.tolist()))

            elif self.model_type == 'XGBClassifier':
                print('Hello')
                display(xgb.explain_weights_xgboost(self.model))

            elif self.model_type == 'LGBMClassifier':
                print('Hello')
                display(lgbm.explain_weights_lightgbm(self.model))

            elif self.model_type == 'CatBoostClassifier':
                display(catb.explain_weights_catboost(self.model))

        elif (
                self.model_type in self.regressors or self.model_type in self.classifiers) and interpretation_tool == 'pdpbox':
            print('\n\nPlotting PDP box interpretations...\n\n')
            ex_intr.Explainer('PDPB')
            for b_feature in self.base_features:
                pdp_goals = pdp.pdp_isolate(model=self.model, dataset=self.X_train, model_features=self.base_features,
                                            feature=b_feature)
                pdp.pdp_plot(pdp_goals, b_feature)
                plt.show()

        elif (
                self.model_type in self.regressors or self.model_type in self.classifiers) and interpretation_tool == 'shap':
            print('\n\nPlotting SHAP interpretations...\n\n')
            shap.initjs()
            explainer = shap.Explainer(self.model)
            shap_values = explainer(self.X_train)
            shap_values.base_value = shap_values.base_values[0]

            print("\n\nPlotting SHAP Summary Plot...\n")
            ex_intr.Explainer('SSP')
            shap.summary_plot(shap_values)

            print("\n\nPlotting SHAP Decision Plot...\n")
            ex_intr.Explainer('SDP')
            shap.decision_plot(shap_values.base_values[0], shap_values.values)

            row_num = randint(0, 100)
            print("\n\nPlotting SHAP Multi-Output Decision Plot for row " + str(row_num) + "...\n")
            ex_intr.Explainer('MODP')
            shap.multioutput_decision_plot([shap_values.base_values.tolist()], [shap_values.values.tolist()], row_num)

            # replace 0 with index of feature to plot
            print("\n\nPlotting SHAP Monitoring Plot...\n")
            shap.monitoring_plot(0, shap_values.values, shap_values.base_values, feature_names=self.base_features)


        elif (self.model_type in self.regressors) and interpretation_tool == 'yellowbrick':
            print("\n\nPlotting Yellowbrick Interpretations...\n\n")

            print("\n\nPlotting Yellowbrick ResidualsPlot...\n")
            ex_intr.Explainer('YBRP')
            visualizer = ResidualsPlot(self.model)
            visualizer.fit(self.X_train, self.y_train)
            visualizer.score(self.X_test, self.y_test)
            visualizer.show()

            print("\n\nPlotting Yellowbrick Prediction Error Plot...\n")
            ex_intr.Explainer('YBPEP')
            prediction_error_visualizer = PredictionError(self.model)
            prediction_error_visualizer.fit(self.X_train, self.y_train)
            prediction_error_visualizer.score(self.X_test, self.y_test)
            prediction_error_visualizer.show()

        elif (self.model_type in self.classifiers) and interpretation_tool == 'yellowbrick':
            print("\n\nPlotting Yellowbrick Interpretations...\n\n")

            print("\n\nPlotting Yellowbrick ClassificationReport...\n")
            visualizer = ClassificationReport(self.model, support=True)
            visualizer.fit(self.X_train, self.y_train)  # Fit the visualizer and the model
            visualizer.score(self.X_test, self.y_test)  # Evaluate the model on the test data
            visualizer.show()

        elif (self.model_type in self.regressors) and interpretation_tool == 'pycaret':
            plots = ['residuals', 'error', 'rfe', 'learning', 'vc', 'manifold', 'parameter', 'feature',
                     'parameter', 'tree']

            print("Plotting PyCaret Interpretations...")
            if(plot_type in plots):
                ex_intr.Explainer(plot_type)
                plot_model(self.model, plot_type)
            else:
                print("No such plot available.")

        elif (self.model_type in self.classifiers) and interpretation_tool == 'pycaret':
            plots = ['auc', 'threshold', 'pr', 'confusion_matrix', 'error', 'class_report', 'boundary', 'rfe', 'learning',
                     'manifold', 'calibration', 'vc', 'dimension', 'feature', 'parameter', 'lift', 'gain', 'tree']

            print("Plotting PyCaret Interpretations...")
            if(plot_type in plots):
                ex_intr.Explainer(str(plot_type))
                plot_model(self.model, plot_type)
            else:
                print("No such plot available.")

In [20]:
class ClassificationMetrics:
    def __init__(self, package, metric_name=None,beta=None,pred_prob=None):
        self.model = package[0]
        self.y_true = package[1]
        self.y_pred = package[2]
        self.beta = beta
        # print(self.y_true)
        # self.metrics_list.append(
        global flag
        flag = 1
        if metric_name is None:
            self._run_all()
        elif metric_name == 'accuracy':
            self._accuracy(self.y_true, self.y_pred)
        # self.metrics_list.append(
        elif metric_name == 'precision':
            self._precision(self.y_true, self.y_pred)
        elif metric_name == 'recall':
            self._recall(self.y_true, self.y_pred)
        elif metric_name == 'f1_score':
            self._f1_score(self.y_true, self.y_pred)
        elif metric_name == 'fbeta_score':
            if beta is not None:
                self._fbeta_score(self.y_true, self.y_pred, beta)
            else:
                print('\033[1m' + '\nF-Beta Score:' + '\033[1m' + '\nBeta Value Required for F-Beta score')
                f_beta = ['F-Beta Score', 'Beta Value Required']
                MetricList.append(f_beta)
        elif metric_name == 'auc':
            self._auc(self.y_true, self.y_pred)
        elif metric_name == 'matthews_corrcoef':
            self._matthews_corrcoef(self.y_true, self.y_pred)
        elif metric_name == 'accuracy':
            self._hamming_loss(self.y_true, self.y_pred)
        elif metric_name == 'hamming_loss':
            self._logloss(self.y_true, self.y_pred)
        elif metric_name == 'zero_one_loss':
            self._zero_one_loss(self.y_true, self.y_pred)
        elif metric_name == 'cohen_kappa_score':
            self._cohen_kappa_score(self.y_true, self.y_pred)
        elif metric_name == 'gini_score':
            if pred_prob is not None:
                self._gini_score(pred_prob)
            else:
                display('Gini Score Required prediction probability')
        else:
            display('Invalid Metric Name')
            flag = 0
            
    def _run_all(self):
        self._accuracy(self.y_true, self.y_pred)
        # self.metrics_list.append(
        self._precision(self.y_true, self.y_pred)
        self._recall(self.y_true, self.y_pred)
        self._f1_score(self.y_true, self.y_pred)
        self._auc(self.y_true, self.y_pred)
        self._matthews_corrcoef(self.y_true, self.y_pred)
        self._hamming_loss(self.y_true, self.y_pred)
        self._logloss(self.y_true, self.y_pred)
        self._zero_one_loss(self.y_true, self.y_pred)
        self._cohen_kappa_score(self.y_true, self.y_pred)

    # @staticmethod
    # def _confusion_matrix(y_true, y_pred):
    #     print('conf')
    #     return metrics.confusion_matrix(y_true, y_pred)

    @staticmethod
    def _gini_score(pred_prob):
        #Logic For Gini Score
        sum_pp=sum(pred_prob)
        val=sum_pp/sum(sum_pp)
        gini = ['Gini Score',1-sum(val*val)]
        MetricList.append(gini)
    
    @staticmethod
    def _accuracy(y_true, y_pred):
        # for i in range(11):
        #     print(y_true[i],"\t",y_pred[i])
        # print(metrics.accuracy_score(y_true, y_pred))
        # print('\033[1m' + '\nAccuracy Score:', metrics.accuracy_score(y_true, y_pred), '\033[0m')
        accuracy = ['Accuracy', metrics.accuracy_score(y_true, y_pred)]
        MetricList.append(accuracy)

    @staticmethod
    def _precision(y_true, y_pred):
        # print('\033[1m' + '\nPrecision:', metrics.precision_score(y_true, y_pred), '\033[0m')
        precision = ['Precision', metrics.precision_score(y_true, y_pred, zero_division=0)]
        MetricList.append(precision)

    @staticmethod
    def _recall(y_true, y_pred):
        # print('\033[1m' + '\nRecall:', metrics.recall_score(y_true, y_pred), '\033[0m')
        recall = ['Recall', metrics.recall_score(y_true, y_pred)]
        MetricList.append(recall)

    @staticmethod
    def _f1_score(y_true, y_pred):
        # print('\033[1m' + '\nF1 Score:', metrics.f1_score(y_true, y_pred), '\033[0m')
        f1_score = ['F1 Score', metrics.f1_score(y_true, y_pred)]
        MetricList.append(f1_score)

    @staticmethod
    def _fbeta_score(y_true, y_pred, beta):
        # print('\033[1m' + '\nF-Beta Score:', metrics.fbeta_score(y_true, y_pred, beta=beta), '\033[0m')
        f_beta = ['F-Beta Score', metrics.fbeta_score(y_true, y_pred, beta=beta)]
        MetricList.append(f_beta)

    @staticmethod
    def _auc(y_true, y_pred):
        # print('\033[1m' + '\nAUC:', metrics.roc_auc_score(y_true, y_pred), '\033[0m')
        auc = ['AUC Score', metrics.roc_auc_score(y_true, y_pred)]
        MetricList.append(auc)

    @staticmethod
    def _classification_report(y_true, y_pred):
        # print('\033[1m' + '\nClassification Report:', metrics.classification_report(y_true, y_pred), '\033[0m')
        class_report = ['Classification Report', metrics.classification_report(y_true, y_pred)]
        MetricList.append(class_report)

    @staticmethod
    def _matthews_corrcoef(y_true, y_pred):
        # print('\033[1m' + '\nMatthews Correlation Coefficient:', metrics.matthews_corrcoef(y_true, y_pred), '\033[0m')
        matthews = ['Matthews CorrCoef', metrics.matthews_corrcoef(y_true, y_pred)]
        MetricList.append(matthews)

    @staticmethod
    def _hamming_loss(y_true, y_pred):
        # print('\033[1m' + '\nHamming Loss:', metrics.hamming_loss(y_true, y_pred), '\033[0m')
        hamming_loss = ['Hamming Loss', metrics.hamming_loss(y_true, y_pred)]
        MetricList.append(hamming_loss)

    @staticmethod
    def _logloss(y_true, y_pred):
        # print('\033[1m' + '\nLog Loss:', metrics.log_loss(y_true, y_pred), '\033[0m')
        log_loss = ['Log Loss', metrics.log_loss(y_true, y_pred)]
        MetricList.append(log_loss)

    @staticmethod
    def _zero_one_loss(y_true, y_pred):
        # print('\033[1m' + '\nZero One Loss:', metrics.zero_one_loss(y_true, y_pred), '\033[0m')
        zero_one_loss = ['Zero One Loss', metrics.zero_one_loss(y_true, y_pred)]
        MetricList.append(zero_one_loss)

    @staticmethod
    def _cohen_kappa_score(y_true, y_pred):
        cohen = ['Cohen Kappa Score', metrics.cohen_kappa_score(y_true, y_pred)]
        MetricList.append(cohen)
        # print('\033[1m' + '\nCohen Kappa:', metrics.cohen_kappa_score(y_true, y_pred), '\033[0m')
    #
    # @staticmethod
    # def _explain(metric_name):


class RegressionMetrics:
    def __init__(self, package, metric_name=None):
        self.model = package[0]
        self.y_true = package[1]
        self.y_pred = package[2]
        global flag
        flag = 1
        if metric_name is None:
            self._run_all()
        elif metric_name == 'explained_variance_score':
            self._explained_variance_score(self.y_true, self.y_pred)
        elif metric_name == 'r2_score':
            self._r2_score(self.y_true, self.y_pred)
        elif metric_name == 'mean_absolute_error':
            self._mean_absolute_error(self.y_true, self.y_pred)
        elif metric_name == 'mean_absolute_percentage_error':
            self._mean_absolute_percentage_error(self.y_true, self.y_pred)
        elif metric_name == 'median_absolute_error':
            self._median_absolute_error(self.y_true, self.y_pred)
        elif metric_name == 'mean_squared_error':
            self._mean_squared_error(self.y_true, self.y_pred)
        elif metric_name == 'root_mean_squared_error':
            self._root_mean_squared_error(self.y_true, self.y_pred)
        elif metric_name == 'mean_squared_log_error':
            self._mean_squared_log_error(self.y_true, self.y_pred)
        elif metric_name == 'root_mean_squared_log_error':
            self._root_mean_squared_log_error(self.y_true, self.y_pred)
        elif metric_name == 'max_error':
            self._max_error(self.y_true, self.y_pred)
        else:
            display('Invalid Metric Name')
            flag = 0

    def _run_all(self):
        self._explained_variance_score(self.y_true, self.y_pred)
        self._r2_score(self.y_true, self.y_pred)
        self._mean_absolute_error(self.y_true, self.y_pred)
        self._mean_absolute_percentage_error(self.y_true, self.y_pred)
        self._median_absolute_error(self.y_true, self.y_pred)
        self._mean_squared_error(self.y_true, self.y_pred)
        self._root_mean_squared_error(self.y_true, self.y_pred)
        self._mean_squared_log_error(self.y_true, self.y_pred)
        self._root_mean_squared_log_error(self.y_true, self.y_pred)
        self._max_error(self.y_true, self.y_pred)

    @staticmethod
    def _explained_variance_score(y_true, y_pred):
        op = ['Explained Variance Score', metrics.explained_variance_score(y_true, y_pred)]
        MetricList.append(op)

    @staticmethod
    def _r2_score(y_true, y_pred):
        op = ['R2 Score', metrics.r2_score(y_true, y_pred)]
        MetricList.append(op)

    @staticmethod
    def _mean_absolute_error(y_true, y_pred):
        op = ['Mean Absolute Error', metrics.mean_absolute_error(y_true, y_pred)]
        MetricList.append(op)

    @staticmethod
    def _mean_absolute_percentage_error(y_true, y_pred):
        x = np.abs(y_true - y_pred) / np.abs(y_true)
        x = (100 / len(y_true)) * x
        x = np.mean(x)
        x = str(x)
        print(type(x))
        op = ['Mean Absolute Percentage Error', x]
        MetricList.append(op)

    @staticmethod
    def _median_absolute_error(y_true, y_pred):
        op = ['Median Absolute Error', metrics.median_absolute_error(y_true, y_pred)]
        MetricList.append(op)

    @staticmethod
    def _mean_squared_error(y_true, y_pred):
        op = ['Mean Squared Error', metrics.mean_squared_error(y_true, y_pred)]
        MetricList.append(op)

    @staticmethod
    def _root_mean_squared_error(y_true, y_pred):
        op = ['Root Mean Squared Error', np.sqrt(metrics.mean_squared_error(y_true, y_pred))]
        MetricList.append(op)

    @staticmethod
    def _mean_squared_log_error(y_true, y_pred):
        op = ['Mean Squared Log Error', metrics.explained_variance_score(y_true, y_pred)]
        MetricList.append(op)

    @staticmethod
    def _root_mean_squared_log_error(y_true, y_pred):
        op = ['Root Mean Squared Log Error', np.sqrt(metrics.mean_squared_log_error(y_true, y_pred))]
        MetricList.append(op)

    @staticmethod
    def _max_error(y_true, y_pred):
        op = ['Max Error', metrics.max_error(y_true, y_pred)]
        MetricList.append(op)


class ClusteringMetrics:
    def __init__(self, package, samples, metric_name=None):
        self.model = package[0]
        self.y_true = package[1]
        self.y_pred = package[2]
        self.samples = samples
        global flag
        flag = 1
        print(metric_name)
        if metric_name is None:
            self._run_all()
        elif metric_name == 'mutual_info_score':
            self._mutual_info_score(self.y_true, self.y_pred)
        elif metric_name == 'normalized_mutual_info_score':
            self._normalized_mutual_info_score(self.y_true, self.y_pred)
        elif metric_name == 'adjusted_mutual_info_score':
            self._adjusted_mutual_info_score(self.y_true, self.y_pred)
        elif metric_name == 'adjusted_rand_score':
            self._adjusted_rand_score(self.y_true, self.y_pred)
        elif metric_name == 'fowlkes_mallows_score':
            self._fowlkes_mallows_score(self.y_true, self.y_pred)
        elif metric_name == 'homogeneity_score':
            self._homogeneity_score(self.y_true, self.y_pred)
        elif metric_name == 'completeness_score':
            self._completeness_score(self.y_true, self.y_pred)
        elif metric_name == 'v_measure_score':
            self._v_measure_score(self.y_true, self.y_pred)
        elif metric_name == 'silhouette_score':
            self._silhouette_score(self.samples, self.y_pred)
        elif metric_name == 'silhouette_samples':
            self._silhouette_samples(self.samples, self.y_pred)
        elif metric_name == 'davies_bouldin_score':
            self._davies_bouldin_score(self.samples, self.y_pred)
        elif metric_name == 'calinski_harabasz_score':
            self._calinski_harabasz_score(self.samples, self.y_pred)
        else:
            display('Invalid Metric Name')
            flag = 0


    def _run_all(self):
        self._mutual_info_score(self.y_true, self.y_pred)
        self._normalized_mutual_info_score(self.y_true, self.y_pred)
        self._adjusted_mutual_info_score(self.y_true, self.y_pred)
        self._adjusted_rand_score(self.y_true, self.y_pred)
        self._fowlkes_mallows_score(self.y_true, self.y_pred)
        self._homogeneity_score(self.y_true, self.y_pred)
        self._completeness_score(self.y_true, self.y_pred)
        self._v_measure_score(self.y_true, self.y_pred)
        self._silhouette_score(self.samples, self.y_pred)
        self._davies_bouldin_score(self.samples, self.y_pred)
        self._calinski_harabasz_score(self.samples, self.y_pred)

    @staticmethod
    def _silhouette_score(y_true, y_pred):
        op = ['Silhouette Score', metrics.silhouette_score(y_true, y_pred)]
        MetricList.append(op)

    @staticmethod
    def _silhouette_samples(y_true, y_pred):
        op = ['Silhouette Sample', metrics.silhouette_samples(y_true, y_pred)]
        MetricList.append(op)

    @staticmethod
    def _mutual_info_score(y_true, y_pred):
        op = ['Mutual Info Score', metrics.mutual_info_score(y_true, y_pred)]
        MetricList.append(op)

    @staticmethod
    def _normalized_mutual_info_score(y_true, y_pred):
        op = ['Normalized Mutual Info Score', metrics.normalized_mutual_info_score(y_true, y_pred)]
        MetricList.append(op)

    @staticmethod
    def _adjusted_mutual_info_score(y_true, y_pred):
        op = ['Adjusted Mutual Info Score', metrics.adjusted_mutual_info_score(y_true, y_pred)]
        MetricList.append(op)

    @staticmethod
    def _adjusted_rand_score(y_true, y_pred):
        op = ['Adjusted Rand Score', metrics.adjusted_rand_score(y_true, y_pred)]
        MetricList.append(op)

    @staticmethod
    def _fowlkes_mallows_score(y_true, y_pred):
        op = ['Fowlkes Mallows Score', metrics.fowlkes_mallows_score(y_true, y_pred)]
        MetricList.append(op)

    @staticmethod
    def _homogeneity_score(y_true, y_pred):
        op = ['Homogeneity Score', metrics.homogeneity_score(y_true, y_pred)]
        MetricList.append(op)

    @staticmethod
    def _completeness_score(y_true, y_pred):
        op = ['Completeness Score', metrics.completeness_score(y_true, y_pred)]
        MetricList.append(op)

    @staticmethod
    def _v_measure_score(y_true, y_pred):
        op = ['V Measure Score', metrics.v_measure_score(y_true, y_pred)]
        MetricList.append(op)

    @staticmethod
    def _davies_bouldin_score(y_true, y_pred):
        op = ['Davies Bouldin Score', metrics.davies_bouldin_score(y_true, y_pred)]
        MetricList.append(op)

    @staticmethod
    def _calinski_harabasz_score(y_true, y_pred):
        op = ['Calinski Harabasz Score', metrics.calinski_harabasz_score(y_true, y_pred)]
        MetricList.append(op)


# print(model,X_test)

In [21]:

#Load Dataset

dataset = pd.read_csv('cars1.csv')
#print(dataset)
y = dataset.selling_price

base_features = ['transmission','year','km_driven','mileage','engine','max_power','seats']

X = dataset[base_features]

#print(X)
#print(y)
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)
first_model = RandomForestRegressor(n_estimators=50, random_state=1).fit(X_train, y_train)

In [22]:
# packaging the data for interpretation
interpretation_package = [first_model, X_train, X_test, y_train, y_test, base_features]
interpreter = Interpreter(interpretation_package)

RandomForestRegressor


In [23]:
#interpreter.interprete_model('eli5')

In [24]:
#interpreter.interprete_model('pdpbox')

In [25]:
#interpreter.interprete_model('shap')

In [26]:
#interpreter.interprete_model('yellowbrick')

In [27]:
interpreter.interprete_model("pycaret", plotname="learning")

model == Random Forest
Model name: RandomForestRegressor
Plotting PyCaret Interpretations...


• Learning Curve: Line plot of learning (y-axis) over experience (x-axis).

• There are three common dynamics that you are likely to observe in learning curves; they are:
		• Underfit.
		• Overfit.
		• Good Fit.

• An underfit model can be identified from the learning curve of the training loss only.
It may show a flat line or noisy values of relatively high loss, indicating that the model was unable to learn the training dataset at all.

• A plot of learning curves shows overfitting if the plot of training loss continues to decrease with experience OR
if the plot of validation loss decreases to a point and begins increasing again.

• A good fit is identified by a training and validation loss that decreases to a point of stability with a minimal gap between
the two final loss values.




ValueError: Plot Not Available. Please see docstring for list of available Plots.